In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading dataset 

# Dense model

NameError: ignored

In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import layers, models

def func(path, label):
    frame = cv2.imread(path)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 2)
    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)

    return thresh, label
# Define the image dimensions and batch size
image_width, image_height = 200, 200
batch_size = 32

# Specify the paths to the directories
train_dir = '/content/drive/Shareddrives/BA865 - Team Project/Image_dataset/train'
validation_dir = '/content/drive/Shareddrives/BA865 - Team Project/Image_dataset/validation'

# Create an ImageDataGenerator for data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and validation data from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 84 images belonging to 34 classes.
Found 34 images belonging to 34 classes.


In [ ]:
# Create the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(220, 220, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluate the model
scores = model.evaluate(validation_dataset)
print(f"Validation accuracy: {scores[1]*100:.2f}%")


Epoch 1/10


InvalidArgumentError: ignored

## Prediction on testset 

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_single_image(model, image_path, target_size, class_labels):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    prediction = model.predict(image_array)
    predicted_class_index = np.argmax(prediction, axis=-1)[0]
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

def predict_multiple_images(model, folder_path, class_labels, target_size=(200, 200)):
    predicted_labels = {}
    for root, dirs, files in os.walk(folder_path):
        for image_filename in files:
            # Construct the full path to the image file
            image_path = os.path.join(root, image_filename)
            if os.path.isfile(image_path):  # Ensure that the item is a file
                predicted_class_label = predict_single_image(model, image_path, target_size, class_labels)
                predicted_labels[image_filename] = predicted_class_label
    return predicted_labels

# Define the path to the folder containing the test images
test_dir = '/content/drive/Shareddrives/BA865 - Team Project/Image_dataset/test'

# Get the class labels from the training generator
class_labels = list(train_generator.class_indices.keys())

# Use the trained model to predict the classes of the test images
predicted_labels = predict_multiple_images(model, test_dir, class_labels)
print('Predicted class labels for test images:')
for image_filename, predicted_class_label in predicted_labels.items():
    print(f'{image_filename}: {predicted_class_label}')


# Conv2 model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the CNN model
model = Sequential([
    # Convolutional layer with 32 filters, each with size 3x3, and ReLU activation
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(200, 200, 3)),
    # Max pooling layer with pool size 2x2
    MaxPooling2D(pool_size=(2, 2)),
    # Dropout layer with 25% dropout rate
    Dropout(0.25),
    # Another convolutional layer with 64 filters, each with size 3x3, and ReLU activation
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    # Max pooling layer with pool size 2x2
    MaxPooling2D(pool_size=(2, 2)),
    # Dropout layer with 25% dropout rate
    Dropout(0.25),
    # Flatten layer to convert 3D feature maps to 1D feature vectors
    Flatten(),
    # Dense layer with 128 neurons and ReLU activation
    Dense(128, activation='relu'),
    # Dropout layer with 50% dropout rate
    Dropout(0.5),
    # Output layer with the number of neurons equal to the number of classes, using softmax activation
    Dense(len(train_generator.class_indices), activation='softmax')  # num_classes should be replaced with the actual number of classes
])


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()


In [ ]:

# Train the model (replace train_generator and validation_generator with your actual generators)
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


In [ ]:

# Evaluate the model on the validation data
scores = model.evaluate(validation_generator)
print(f"Validation accuracy: {scores[1]*100:.2f}%")


## Prediction on testset

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_single_image(model, image_path, target_size, class_labels):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    prediction = model.predict(image_array)
    predicted_class_index = np.argmax(prediction, axis=-1)[0]
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

def predict_multiple_images(model, folder_path, class_labels, target_size=(200, 200)):
    predicted_labels = {}
    for root, dirs, files in os.walk(folder_path):
        for image_filename in files:
            # Construct the full path to the image file
            image_path = os.path.join(root, image_filename)
            if os.path.isfile(image_path):  # Ensure that the item is a file
                predicted_class_label = predict_single_image(model, image_path, target_size, class_labels)
                predicted_labels[image_filename] = predicted_class_label
    return predicted_labels

# Define the path to the folder containing the test images
test_dir = '/content/drive/Shareddrives/BA865 - Team Project/Image_dataset/test'

# Get the class labels from the training generator (replace train_generator with your actual generator)
class_labels = list(train_generator.class_indices.keys())



In [ ]:
# Use the trained model to predict the classes of the test images
predicted_labels = predict_multiple_images(model, test_dir, class_labels)
print('Predicted class labels for test images:')
for image_filename, predicted_class_label in predicted_labels.items():
    print(f'{image_filename}: {predicted_class_label}')